In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
data=pd.read_csv("data/ImmGenATAC18_AllOCRsInfo.csv"
                 )

cols = ["ILC2.SI", "ILC3.NKp46-CCR6-.SI", "ILC3.NKp46+.SI", "ILC3.CCR6+.SI",
        "NK.27+11b-.BM", "NK.27+11b+.BM", "NK.27-11b+.BM",
        "NK.27+11b-.Sp", "NK.27+11b+.Sp", "NK.27-11b+.Sp"]
subset = data[cols]

# 3. Transponieren: Zelltypen werden zu Zeilen
data_transposed = subset.T
data_transposed.index.name = "CellType"

# Jetzt kannst du mit `data_transposed` weiterarbeiten
print(data_transposed.head())

#data_sampled = data.sample(n=5000, random_state=42)
#Daten standardisieren (z-Score pro OCR):
#scaler = StandardScaler()
#data_scaled = scaler.fit_transform(data_sampled)
#data_scaled
#PCA

pca = PCA(n_components=25)
pca_scores = PCA().fit_transform(data_transposed)
# create a dataframe of pca_scores
df_pc = pd.DataFrame(pca_scores)

# create a dataframe of pca_scores
df_pc = pd.DataFrame(pca_scores)
df_pc

# perform t-SNE on PCs scores
# we will use first 50 PCs but this can vary
from sklearn.manifold import TSNE
#tsne_em = TSNE(n_components = 2, perplexity = 30.0, early_exaggeration = 12, 
 #              n_iter = 1000, learning_rate = 368, verbose = 1).fit_transform(df_pc[:,0:49])
tsne = TSNE(n_components=2, perplexity=50, n_iter=500, learning_rate=200, verbose=1)
tsne_result = tsne.fit_transform(df_pc)

###questions so far: is it okay to reduce so radical to a sample of 5000? 
###and how many PCs used?
###and how optimize t-SNE parameters perplexicity and n_iter?
###for visualization: does it look richt??? for eps>3 it is just a big blob, for <3 it is big blob and small dots???
###also what for min_samples
#transpose > pca > sample > tsne > heirarchichal clusetr > color by color
#read me: look at hhow different papers do readmes
from bioinfokit.visuz import cluster
from sklearn.cluster import DBSCAN
cluster.tsneplot(score=tsne_result, show=True)
get_clusters = DBSCAN(eps = 1, min_samples = 10).fit_predict(tsne_result)
print(set(get_clusters))

cluster.tsneplot(score=tsne_result, colorlist=[str(i) for i in get_clusters], 
    colordot=('#713e5a', '#63a375', '#edc79b', '#d57a66', '#ca6680', '#395B50', '#92AFD7', '#b0413e', '#4381c1', '#736ced', '#631a86', '#de541e', '#022b3a', '#000000'), 
    legendpos='upper right', legendanchor=(1.15, 1), show=True)


hierarchical_clustering = AgglomerativeClustering(n_clusters=10)  # Adjust n_clusters as needed
cluster_labels = hierarchical_clustering.fit_predict(tsne_result)

# Step 4: Visualize the t-SNE results with clusters
plt.figure(figsize=(8, 6))
for cluster in np.unique(cluster_labels):
    plt.scatter(tsne_result[cluster_labels == cluster, 0],
                tsne_result[cluster_labels == cluster, 1],
                label=f'Cluster {cluster}')
plt.title("t-SNE with Hierarchical Clustering")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend()
plt.show()

# here eps parameter is very important and optimizing eps is essential
# for well defined clusters. I have run DBSCAN with several eps values
# and got good clusters with eps=3
#get_clusters = DBSCAN(eps = 2, min_samples = 10).fit_predict(tsne_result)
# check unique clusters
# -1 value represents noisy points could not assigned to any cluster
#set(get_clusters)
#print(set(get_clusters))
# get t-SNE plot with colors assigned to each cluster
#transpose matrix!
#leiden, k means: ists supposed to overlap, with matplotlib, splt,color by cell type and cluster
#

#



                     0       1       2       3       4       5       6       \
CellType                                                                      
ILC2.SI                1.79    1.29    0.44    1.76    1.43    1.76    0.44   
ILC3.NKp46-CCR6-.SI    1.67    3.87    2.49    0.95    0.15    0.57    1.36   
ILC3.NKp46+.SI         0.75    0.23    0.23    0.23    2.14    0.23    0.23   
ILC3.CCR6+.SI          1.21    0.15    0.95    0.15    0.15    0.95    1.41   
NK.27+11b-.BM          0.82    1.46    0.20    1.72    0.20    1.46    0.82   

                     7       8       9       ...  512585  512586  512587  \
CellType                                     ...                           
ILC2.SI                0.10    1.29    1.79  ...   15.72    6.77    7.65   
ILC3.NKp46-CCR6-.SI    3.98    3.14    1.36  ...   11.72    8.04    1.93   
ILC3.NKp46+.SI         0.23    0.75    0.75  ...   16.99    8.56    7.52   
ILC3.CCR6+.SI          1.41    0.95    0.62  ...   15.15    3.59  

c:\Users\janna\miniconda3\envs\data_analysis\Lib\site-packages\sklearn\manifold\_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


ValueError: perplexity must be less than n_samples